<a href="https://colab.research.google.com/github/motallebimasoud-cmyk/binance-volume-bot/blob/main/Volume7_30min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
from datetime import datetime

# -------------------------------
# تنظیمات تلگرام
# -------------------------------
TELEGRAM_TOKEN = "8421847126:AAF8DyYSO76IBzQ8IY7tV7SC0qzzcT6-T20"
CHAT_ID = "953826363"

def send_telegram(message: str):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    data = {"chat_id": CHAT_ID, "text": message}
    try:
        requests.post(url, data=data, timeout=10)
    except Exception as e:
        print("❌ Telegram Error:", e)

# -------------------------------
# گرفتن لیست همه مارکت‌های USDT از KuCoin
# -------------------------------
def get_markets():
    url = "https://api.kucoin.com/api/v1/symbols"
    try:
        r = requests.get(url, timeout=10).json()
        if r["code"] == "200000":
            return [m["symbol"] for m in r["data"] if m["symbol"].endswith("USDT")]
    except Exception as e:
        print("❌ Market Error:", e)
    return []

# -------------------------------
# داده کندل‌ها
# -------------------------------
def get_klines(symbol, interval="30min", limit=21):
    url = f"https://api.kucoin.com/api/v1/market/candles"
    params = {"symbol": symbol, "type": interval}
    try:
        r = requests.get(url, params=params, timeout=10).json()
        if r["code"] == "200000":
            return r["data"][:limit][::-1]  # آخرین کندل‌ها (مرتب از قدیم به جدید)
    except Exception as e:
        print(f"❌ Kline Error {symbol}:", e)
    return []

# -------------------------------
# چک کردن شرایط حجم + ارزش معاملات + رشد قیمت
# -------------------------------
def check_symbol(symbol):
    klines = get_klines(symbol)
    if not klines or len(klines) < 21:
        return

    # کندل‌ها
    volumes = [float(k[5]) for k in klines]
    last_volume = volumes[-1]
    avg_volume = sum(volumes[:-1]) / 20

    last_turnover = float(klines[-1][6])  # ارزش معاملات
    last_open = float(klines[-1][1])      # قیمت باز شدن
    last_close = float(klines[-1][2])     # قیمت بسته شدن

    # درصد تغییر قیمت
    price_change_pct = ((last_close - last_open) / last_open) * 100

    # شرایط
    if last_volume > avg_volume * 2.5 and last_turnover > 100000 and price_change_pct >= 1:
        msg = (
            f"📊 KuCoin Alert\n"
            f"📈 حجم غیرعادی در {symbol}\n"
            f"⏰ تایم‌فریم: 30 دقیقه\n"
            f"حجم: {last_volume:,.0f}\n"
            f"میانگین 20 کندل: {avg_volume:,.0f}\n"
            f"💰 ارزش معاملات: {last_turnover:,.0f} USDT\n"
            f"📊 تغییر قیمت: {price_change_pct:.2f}%\n"
            f"🔗 https://www.kucoin.com/trade/{symbol.replace('-', '_')}"
        )
        send_telegram(msg)
        print(f"✅ Alert sent for {symbol} | {price_change_pct:.2f}% | {last_turnover:,.0f} USDT")

# -------------------------------
# اجرای همیشگی (۳۰ دقیقه‌ای)
# -------------------------------
while True:
    try:
        now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
        markets = get_markets()
        print(f"\n⏰ {now} | تعداد مارکت‌های USDT پیدا شد: {len(markets)}")
        for symbol in markets:
            check_symbol(symbol)
        print("🔎 چک همه مارکت‌ها انجام شد ✅")
    except Exception as e:
        print("❌ Error:", e)

    time.sleep(1800)   # هر ۳۰ دقیقه

/tmp/ipython-input-505546070.py:86: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")



⏰ 2025-09-22 17:00:14 | تعداد مارکت‌های USDT پیدا شد: 1080
🔎 چک همه مارکت‌ها انجام شد ✅

⏰ 2025-09-22 17:35:30 | تعداد مارکت‌های USDT پیدا شد: 1080
🔎 چک همه مارکت‌ها انجام شد ✅

⏰ 2025-09-22 18:10:38 | تعداد مارکت‌های USDT پیدا شد: 1080
🔎 چک همه مارکت‌ها انجام شد ✅
